In [11]:
from ctypes import *
import math
import random
import os
import cv2
import numpy as np
import time
import json

import darknet

from datetime import datetime
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
FLAGS = {
    'HELMET_DRAW_ENABLED' : True,
    'SAVE_ON_NEW_HEAD' : True,
    'SHOW_ORIGINAL_IMAGE' : True,
    'SHOW_FPS' : True
}

def resizeDetections(original_image_shape, network_image_size, detections):
    resize_ratio = (original_image_shape[1]/network_image_size[0], original_image_shape[0]/network_image_size[1])
    resized_detections = []
    for detection in detections:
        resized_detections.append(
            (
                detection[0],
                detection[1],
                (
                detection[2][0] * resize_ratio[0],
                detection[2][1] * resize_ratio[1],
                detection[2][2] * resize_ratio[0],
                detection[2][3] * resize_ratio[1]
                )
            )
        )
    return resized_detections
    

def convertBack(x, y, w, h):
    xmin = int(round(x - (w / 2)))
    xmax = int(round(x + (w / 2)))
    ymin = int(round(y - (h / 2)))
    ymax = int(round(y + (h / 2)))
    return xmin, ymin, xmax, ymax

def cvDrawBoxes(detections, img):
    for detection in detections:
        x, y, w, h = detection[2][0],\
            detection[2][1],\
            detection[2][2],\
            detection[2][3]
        xmin, ymin, xmax, ymax = convertBack(
            float(x), float(y), float(w), float(h))
        pt1 = (xmin, ymin)
        pt2 = (xmax, ymax)
        if (detection[0].decode() == "head"):
            cv2.rectangle(img, pt1, pt2, (255, 0, 0), 1)
            cv2.putText(img,
                        detection[0].decode() +
                        " [" + str(round(detection[1] * 100, 2)) + "]",
                        (pt1[0], pt1[1] - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                        [255, 0, 0], 2)
        else:
            cv2.rectangle(img, pt1, pt2, (0, 255, 0), 1)
            cv2.putText(img,
                        detection[0].decode() +
                        " [" + str(round(detection[1] * 100, 2)) + "]",
                        (pt1[0], pt1[1] - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                        [0, 255, 0], 2)
    return img


In [3]:
def drawImage(img, tofile=False, num=0):
    
    if (tofile):
        cv2.imwrite("saved-%d.jpg" % num, img)
    else:
        pilimg = Image.fromarray(img)
        display(pilimg)
        
def saveImage(img, num=0):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    cv2.imwrite("saved-%d.jpg" % num, img)

In [4]:
netMain = None
metaMain = None
altNames = None

In [5]:
def detect(image):

    global metaMain, netMain, altNames
    
    configPath = "./yolov4-helmet-detection.cfg"
    weightPath = "./yolov4-helmet-detection.weights"
    metaPath = "./yolov4-helmet-detection.data"

    if netMain is None:
        netMain = darknet.load_net_custom(configPath.encode("ascii"), weightPath.encode("ascii"), 0, 1)  # batch size = 1
    if metaMain is None:
        metaMain = darknet.load_meta(metaPath.encode("ascii"))

    # Create an image we reuse for each detect
    darknet_image = darknet.make_image(darknet.network_width(netMain),
                                    darknet.network_height(netMain),3)

    darknet.copy_image_from_bytes(darknet_image, image.tobytes())

    detections = darknet.detect_image(netMain, metaMain, darknet_image, thresh=0.25)

    return detections

In [6]:
imagepath = "sample2.jpg"

In [7]:
img = cv2.imread(imagepath)

image_size = img.shape
network_image_size = (416, 416)
image_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# network image size (416*416, ...)
image_resized = cv2.resize(image_rgb, (416, 416), interpolation=cv2.INTER_LINEAR)

#saveImage(image_resized, 2)

In [8]:
%%time

detections = detect(image_resized)

CPU times: user 2.24 s, sys: 746 ms, total: 2.99 s
Wall time: 4.11 s


In [9]:
for d in detections:
    print(d[0], d[1], d[2])

b'helmet' 0.9973575472831726 (117.57293701171875, 176.67514038085938, 54.27720260620117, 128.32260131835938)
b'helmet' 0.9905349612236023 (177.47865295410156, 204.83558654785156, 47.36790466308594, 105.55615234375)
b'helmet' 0.9709359407424927 (240.6058349609375, 188.94381713867188, 50.24748611450195, 105.96436309814453)


In [17]:
type(detections[0][0])

bytes

In [20]:
results = []
for d in detections:
    r = {
        "type": "entity",
        "entity": {
            "tag": {
                "value" : d[0].decode('utf-8'),
                "confidence": round(d[1], 4),
            },
            "box" : {
                "l" : int(d[2][0]),
                "t" : int(d[2][1]),
                "w" : int(d[2][2]),
                "h" : int(d[2][3])
            }
        }
    }
    results.append(r)
    
respBody = {
    "inferences" : results
}

respBody = json.dumps(respBody)
print(respBody)

{"inferences": [{"type": "entity", "entity": {"tag": {"value": "helmet", "confidence": 0.9974}, "box": {"l": 117, "t": 176, "w": 54, "h": 128}}}, {"type": "entity", "entity": {"tag": {"value": "helmet", "confidence": 0.9905}, "box": {"l": 177, "t": 204, "w": 47, "h": 105}}}, {"type": "entity", "entity": {"tag": {"value": "helmet", "confidence": 0.9709}, "box": {"l": 240, "t": 188, "w": 50, "h": 105}}}]}


In [ ]:
detections = resizeDetections(image_size, network_image_size, detections)

detect_image = cvDrawBoxes(detections, image_rgb)

In [ ]:
drawImage(detect_image)

In [ ]:
type(detect)

In [ ]:
netMain = None
metaMain = None
altNames = None

def YOLO(imagepath):

    global metaMain, netMain, altNames
    
    configPath = "./configs/yolov4-helmet-detection.cfg"
    weightPath = "./configs/yolov4-helmet-detection.weights"
    metaPath = "./configs/yolov4-helmet-detection.data"

    if not os.path.exists(configPath):
        raise ValueError("Invalid config path `" +
                         os.path.abspath(configPath)+"`")
    if not os.path.exists(weightPath):
        raise ValueError("Invalid weight path `" +
                         os.path.abspath(weightPath)+"`")
    if not os.path.exists(metaPath):
        raise ValueError("Invalid data file path `" +
                         os.path.abspath(metaPath)+"`")
    if netMain is None:
        netMain = darknet.load_net_custom(configPath.encode(
            "ascii"), weightPath.encode("ascii"), 0, 1)  # batch size = 1
    if metaMain is None:
        metaMain = darknet.load_meta(metaPath.encode("ascii"))
    if altNames is None:
        try:
            with open(metaPath) as metaFH:
                metaContents = metaFH.read()
                import re
                match = re.search("names *= *(.*)$", metaContents,
                                  re.IGNORECASE | re.MULTILINE)
                if match:
                    result = match.group(1)
                else:
                    result = None
                try:
                    if os.path.exists(result):
                        with open(result) as namesFH:
                            namesList = namesFH.read().strip().split("\n")
                            altNames = [x.strip() for x in namesList]
                except TypeError:
                    pass
        except Exception:
            pass

    if not os.path.exists("outputs"):
        os.mkdir("outputs")

    # load video file / streams
    image = cv2.imread(imagepath)

    print("Starting the YOLO loop...")

    # Create an image we reuse for each detect
    darknet_image = darknet.make_image(darknet.network_width(netMain),
                                    darknet.network_height(netMain),3)
    # network image size (416*416, ...)
    network_image_size = (darknet.network_width(netMain),
                          darknet.network_height(netMain))
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_resized = cv2.resize(image_rgb,
                               network_image_size,
                               interpolation=cv2.INTER_LINEAR)

    darknet.copy_image_from_bytes(darknet_image, image_resized.tobytes())

    detections = darknet.detect_image(netMain, metaMain, darknet_image, thresh=0.25)
    detections = resizeDetections(image.shape, network_image_size, detections)
    
    detect_image = cvDrawBoxes(detections, image_rgb)
    detect_image = cv2.cvtColor(detect_image, cv2.COLOR_BGR2RGB)

    cv2.imshow("detected", detect_image)
    cv2.imwrite("output.jpg", detect_image)

    while True:
        # press 'q' to quit
        if cv2.waitKey(1) == ord('q'):
            break
    cv2.destroyAllWindows()


## app test

In [ ]:
!curl -X POST -H "Content-type: image/jpeg" --data-binary @"saved-1.jpg" localhost:8089/score